# Scrapping the Neighborhood
### Capstone project, Week 3, task1

The task is to get the neighborhood information from wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

First of all, let's import packages that are commonly used for web scrapping. 
This list includes requests, urlib.request, time, bs4
We also need to import pandas

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

Use requests to get the content from the url

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
response #if the connection is good then the result should be <Response [200]>

<Response [200]>

Use beautiful soup to parse the page. We only need the first table that contains neighborhood information.

In [73]:
soup = BeautifulSoup(response.text, "html.parser")
table = soup.findAll('table')[0] #only parse the first table

convert the tabel into a dataframe and add the headings

In [75]:
df = []
for rows in table.findAll('tr'):
    row = []
    for items in rows.findAll('td'):
        row.append(items.text.replace('\n',''))
    df.append(row)
df.pop(0)
df = pd.DataFrame(df, columns=['Postcode', 'Borough', 'Neighborhood'])
print(df.head(),"\n\n", df.shape)

  Postcode           Borough      Neighborhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront 

 (288, 3)


Remove the rows with Borough or Neighborhood being "Not assigned"

In [80]:
df = df[(df['Borough'] != 'Not assigned') & (df['Neighborhood'] != 'Not assigned')]
print("Any boroughs unassigned? ",(df['Borough'] == 'Not assigned').value_counts())
print("Any neighborhoods unassigned? ",(df['Neighborhood'] == 'Not assigned').value_counts())

Any boroughs unassigned?  False    102
Name: Borough, dtype: int64
Any neighborhoods unassigned?  False    102
Name: Neighborhood, dtype: int64


Combine neighborhoods with the same postcode.

In [81]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].agg(lambda col: ','.join(col)).reset_index()

Show the shape of cleaned dataframe

In [83]:
df.shape

(102, 3)